# Hyperparameter optimization

## Problem definition

Running a neural network is complicated, however finding a convenient (or, *let be optimistic* optimal) set of hyperparameters is even harder.

By *hyperparameter* we mean all the values that define the run. They can be related to the network structure, or to the training process:
- number of layers, number of neurons (more generally, network structure)
- batch size
- image size
- optimizer
- learning rate (starting learning rate, decay step, decay rate)
- ...

## Solution

To address the hyperparameter optimization in this notebook, we use `hyperas`, a [Python library](https://github.com/maxpumperla/hyperas) that wraps [hyperopt](https://github.com/hyperopt/hyperopt), another famous Python library dedicated to parameter optimization.

## Code

### Imports

First we have to import the needed dependencies:

In [1]:
from hyperopt import STATUS_OK, tpe, Trials
from hyperas import optim

/home/rde/.envs/cnn/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
from keras.models import Model
from keras.optimizers import Adam

Some other dependencies are related to our own code:

In [3]:
from deeposlandia.generator import create_generator
from deeposlandia.keras_feature_detection import FeatureDetectionNetwork

### Data declaration

In [29]:
def data():
    """
    """
    DATASET = "shapes"
    MODEL = "feature_detection"
    IMAGE_SIZE = 64
    DATAPATH = os.path.join("..", "data", DATASET, "preprocessed", str(IMAGE_SIZE) + "_full")
    BATCH_SIZE = 50
    LABELS = range(3)
    SEED = 1337
    train_generator = create_generator(dataset=DATASET,
                                       model=MODEL,
                                       datapath=os.path.join(DATAPATH, "training"), 
                                       image_size=IMAGE_SIZE,
                                       batch_size=BATCH_SIZE,
                                       label_ids=LABELS,
                                       seed=SEED)
    validation_generator = create_generator(dataset=DATASET,
                                            model=MODEL,
                                            datapath=os.path.join(DATAPATH, "validation"),
                                            image_size=IMAGE_SIZE,
                                            batch_size=BATCH_SIZE,
                                            label_ids=LABELS,
                                            seed=SEED)
    testing_generator = create_generator(dataset=DATASET,
                                         model=MODEL,
                                         datapath=os.path.join(DATAPATH, "testing"),
                                         image_size=IMAGE_SIZE,
                                         batch_size=BATCH_SIZE,
                                         label_ids=LABELS,
                                         seed=SEED,
                                         inference=True)
    return train_generator, validation_generator, testing_generator

In [5]:
def data_unbatch():
    """
    """
    DATASET = "shapes"
    MODEL = "feature_detection"
    IMAGE_SIZE = 64
    DATAPATH = os.path.join("..", "data", DATASET, "preprocessed", str(IMAGE_SIZE) + "_full")
    BATCH_SIZE = 50
    LABELS = range(3)
    SEED = 1337
    train_generator = create_generator(dataset=DATASET,
                                       model=MODEL,
                                       datapath=os.path.join(DATAPATH, "training"), 
                                       image_size=IMAGE_SIZE,
                                       batch_size=BATCH_SIZE,
                                       label_ids=LABELS,
                                       seed=SEED)
    validation_generator = create_generator(dataset=DATASET,
                                            model=MODEL,
                                            datapath=os.path.join(DATAPATH, "validation"),
                                            image_size=IMAGE_SIZE,
                                            batch_size=BATCH_SIZE,
                                            label_ids=LABELS,
                                            seed=SEED)
    testing_generator = create_generator(dataset=DATASET,
                                         model=MODEL,
                                         datapath=os.path.join(DATAPATH, "testing"),
                                         image_size=IMAGE_SIZE,
                                         batch_size=BATCH_SIZE,
                                         label_ids=LABELS,
                                         seed=SEED)
    train_X, train_Y = next(train_generator)
    val_X, val_Y = next(validation_generator)
    test_X = next(testing_generator)
    return train_X, train_Y, val_X, val_Y, test_X

### Model declaration

In [6]:
def create_model(train_generator, validation_generator):
    """
    """
    IMAGE_SIZE = 64
    BATCH_SIZE = 50
    NB_EPOCHS = 10
    NB_TRAINING_IMAGES = 1000
    NB_VALIDATION_IMAGES = 200
    cnn = FeatureDetectionNetwork("test", image_size=IMAGE_SIZE, nb_labels=len(LABELS))
    model = Model(cnn.X, cnn.Y)
    model.compile(loss='binary_crossentropy',
                  optimizer={{choice(['adam', 'sgd', 'adagrad'])}},
                  metrics=['acc'])
    model.fit_generator(train_generator,
                        epochs=NB_EPOCHS,
                        steps_per_epoch=NB_TRAINING_IMAGES // BATCH_SIZE)
    score, acc = model.evaluate_generator(validation_generator, steps=NB_VALIDATION_IMAGES // BATCH_SIZE)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [7]:
def create_model_unbatch(train_X, train_Y, val_X, val_Y):
    """
    """
    IMAGE_SIZE = 64
    BATCH_SIZE = 50
    NB_EPOCHS = 10
    NB_TRAINING_IMAGES = 1000
    NB_VALIDATION_IMAGES = 200
    cnn = FeatureDetectionNetwork("test", image_size=IMAGE_SIZE, nb_labels=len(LABELS))
    model = Model(cnn.X, cnn.Y)
    adam = Adam(lr={{choice(0.01, 0.001, 0.0001)}}, decay={{choice(1e-3, 1e-4, 1e-5)}})
    model.compile(loss='binary_crossentropy',
                  optimizer=optim,
                  metrics=['acc'])
    model.fit(x=train_X, y=train_Y, epochs=NB_EPOCHS,
                        steps_per_epoch=NB_TRAINING_IMAGES // BATCH_SIZE)
    score, acc = model.predict(x=val_X ,y=val_Y, steps=NB_VALIDATION_IMAGES // BATCH_SIZE)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}    

### Execution

In [30]:
train_generator, validation_generator, test_generator = data()

Found 18000 images belonging to 1 classes.
Found 18000 images belonging to 1 classes.
Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.
Found 5000 images belonging to 1 classes.


In [9]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='hyperparam_optimization')

>>> Imports:
#coding=utf-8

try:
    from hyperopt import STATUS_OK, tpe, Trials
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    import os
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam
except:
    pass

try:
    from deeposlandia.generator import create_generator
except:
    pass

try:
    from deeposlandia.keras_feature_detection import FeatureDetectionNetwork
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'optimizer': hp.choice('optimizer', ['adam', 'sgd', 'adagrad']),
    }

>>> Data
   1: 
   2: """
   3: """
   4: DATASET = "shapes"
   5: MODEL = "feature_detection"
   6: IMAGE_SIZE = 64
   7: DATAPATH = os.path.join("..", "data", DATASET, "preprocessed", str(IMAGE_SIZE) + "_full")
   8: BATCH_SIZE = 50
   9: LABELS = range(3)
  10: SEED = 1337
  11: train_generator = create_generator(dataset=DATASET,
  12:                                    

20/20 [==============================] - 6s 311ms/step - loss: 0.6742 - acc: 0.6717


In [31]:
NB_TESTING_IMAGES = 5000
BATCH_SIZE = 50
best_model.predict_generator(test_generator, steps=NB_TESTING_IMAGES // BATCH_SIZE)

array([[0.44647166, 0.42901668, 0.30491966],
       [0.58880764, 0.83338577, 0.94683397],
       [0.67029434, 0.577556  , 0.49348247],
       ...,
       [0.43205374, 0.51706254, 0.62923896],
       [0.80425006, 0.68572044, 0.7020138 ],
       [0.59167844, 0.5832041 , 0.61971694]], dtype=float32)